# Pre-processing training data

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load training data set from CSV file
training_data_df = pd.read_csv("sales_data_training.csv")

# Load testing data set from CSV file
test_data_df = pd.read_csv("sales_data_test.csv")

# Data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well.
scaler = MinMaxScaler(feature_range=(0,1))

# Scale both the training inputs and outputs
scaled_training = scaler.fit_transform(training_data_df)
scaled_testing = scaler.transform(test_data_df)

# Print out the adjustment that the scaler applied to the total_earnings column of data
print("Note: total_earnings values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[8], scaler.min_[8]))

# Create new pandas DataFrame objects from the scaled data
scaled_training_df = pd.DataFrame(scaled_training, columns=training_data_df.columns.values)
scaled_testing_df = pd.DataFrame(scaled_testing, columns=test_data_df.columns.values)

# Save scaled data dataframes to new CSV files
scaled_training_df.to_csv("sales_data_training_scaled.csv", index=False)
scaled_testing_df.to_csv("sales_data_test_scaled.csv", index=False)

Note: total_earnings values were scaled by multiplying by 0.0000036968 and adding -0.115913


# Define a Keras model using the Sequential API

In [3]:
import pandas as pd
from keras.models import Sequential
from keras.layers import *

training_data_df = pd.read_csv("sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

# Define the model
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Traing and Evaluation the model

In [4]:
import pandas as pd
from keras.models import Sequential
from keras.layers import *

training_data_df = pd.read_csv("sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

# Define the model
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X, Y, epochs=50, shuffle=True, verbose=2)

# Load the separate test data set
test_data_df = pd.read_csv("sales_data_test_scaled.csv")

X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

Epoch 1/50
32/32 - 0s - loss: 0.0689
Epoch 2/50
32/32 - 0s - loss: 0.0076
Epoch 3/50
32/32 - 0s - loss: 0.0022
Epoch 4/50
32/32 - 0s - loss: 8.7547e-04
Epoch 5/50
32/32 - 0s - loss: 5.0223e-04
Epoch 6/50
32/32 - 0s - loss: 3.4950e-04
Epoch 7/50
32/32 - 0s - loss: 2.4626e-04
Epoch 8/50
32/32 - 0s - loss: 1.8757e-04
Epoch 9/50
32/32 - 0s - loss: 1.5020e-04
Epoch 10/50
32/32 - 0s - loss: 1.3380e-04
Epoch 11/50
32/32 - 0s - loss: 1.0592e-04
Epoch 12/50
32/32 - 0s - loss: 9.0066e-05
Epoch 13/50
32/32 - 0s - loss: 7.9995e-05
Epoch 14/50
32/32 - 0s - loss: 7.1543e-05
Epoch 15/50
32/32 - 0s - loss: 6.7761e-05
Epoch 16/50
32/32 - 0s - loss: 6.0414e-05
Epoch 17/50
32/32 - 0s - loss: 5.2848e-05
Epoch 18/50
32/32 - 0s - loss: 4.8212e-05
Epoch 19/50
32/32 - 0s - loss: 5.0350e-05
Epoch 20/50
32/32 - 0s - loss: 3.9775e-05
Epoch 21/50
32/32 - 0s - loss: 4.2375e-05
Epoch 22/50
32/32 - 0s - loss: 3.6324e-05
Epoch 23/50
32/32 - 0s - loss: 3.2881e-05
Epoch 24/50
32/32 - 0s - loss: 3.3314e-05
Epoch 25/50
3

# Making perdictions

In [5]:
# Load the data we make to use to make a prediction
X = pd.read_csv("proposed_new_product.csv").values

# Make a prediction with the neural network
prediction = model.predict(X)

# Grab just the first element of the first prediction (since that's the only have one)
prediction = prediction[0][0]

# Re-scale the data from the 0-to-1 range back to dollars
# These constants are from when the data was originally scaled down to the 0-to-1 range
prediction = prediction + 0.1159
prediction = prediction / 0.0000036968

print("Earnings Prediction for Proposed Product - ${}".format(prediction))


Earnings Prediction for Proposed Product - $266108.9631443647


# Saving and loading models

In [6]:
# Save the model to disk
model.save('trained_model.h5')
print("Model saved to disk.")

Model saved to disk.


In [7]:
from keras.models import load_model

model = load_model("trained_model.h5")

X = pd.read_csv("proposed_new_product.csv").values
prediction = model.predict(X)

# Grab just the first element of the first prediction (since we only have one)
prediction = prediction[0][0]

# Re-scale the data from the 0-to-1 range back to dollars
# These constants are from when the data was originally scaled down to the 0-to-1 range
prediction = prediction + 0.1159
prediction = prediction / 0.0000036968

print("Earnings Prediction for Proposed Product - ${}".format(prediction))


Earnings Prediction for Proposed Product - $266108.9631443647


# Recognize images with ResNet50 model

In [8]:
import numpy as np
from keras.preprocessing import image
from keras.applications import resnet50

# Load Keras' ResNet50 model that was pre-trained against the ImageNet database
model = resnet50.ResNet50()

# Load the image file, resizing it to 224x224 pixels (required by this model)
img = image.load_img("bay.jpg", target_size=(224, 224))

# Convert the image to a numpy array
x = image.img_to_array(img) 

# Add a forth dimension since Keras expects a list of images
x = np.expand_dims(x, axis=0)

# Scale the input image to the range used in the trained network
x = resnet50.preprocess_input(x)

# Run the image through the deep neural network to make a prediction
predictions = model.predict(x)

# Look up the names of the predicted classes. Index zero is the results for the first image.
predicted_classes = resnet50.decode_predictions(predictions, top=9)

print("This is an image of:")

for imagenet_id, name, likelihood in predicted_classes[0]:
    print(" - {}: {:2f} likelihood".format(name, likelihood))



40960/35363 [==================================] - 0s 9us/step
This is an image of:
 - seashore: 0.570416 likelihood
 - lakeside: 0.292265 likelihood
 - dock: 0.082964 likelihood
 - breakwater: 0.037219 likelihood
 - promontory: 0.005919 likelihood
 - catamaran: 0.002877 likelihood
 - sandbar: 0.001334 likelihood
 - trimaran: 0.000754 likelihood
 - pier: 0.000743 likelihood


In [ ]:
# Export Keras logs in TensorFlow format

In [9]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import *

training_data_df = pd.read_csv("sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

# Define the model
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu', name='layer_1'))
model.add(Dense(100, activation='relu', name='layer_2'))
model.add(Dense(50, activation='relu', name='layer_3'))
model.add(Dense(1, activation='linear', name='output_layer'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Create a TensorBoard logger
logger = keras.callbacks.TensorBoard(
    log_dir="logs",
    write_graph=True,
    histogram_freq=5,
)

# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2,
    callbacks=[logger]
)

# Load the separate test data set
test_data_df = pd.read_csv("sales_data_test_scaled.csv")

X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

Epoch 1/50
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
32/32 - 0s - loss: 0.0310
Epoch 2/50
32/32 - 0s - loss: 0.0037
Epoch 3/50
32/32 - 0s - loss: 0.0011
Epoch 4/50
32/32 - 0s - loss: 5.6026e-04
Epoch 5/50
32/32 - 0s - loss: 3.1208e-04
Epoch 6/50
32/32 - 0s - loss: 2.1780e-04
Epoch 7/50
32/32 - 0s - loss: 1.4152e-04
Epoch 8/50
32/32 - 0s - loss: 1.3207e-04
Epoch 9/50
32/32 - 0s - loss: 1.0002e-04
Epoch 10/50
32/32 - 0s - loss: 7.6446e-05
Epoch 11/50
32/32 - 0s - loss: 6.3419e-05
Epoch 12/50
32/32 - 0s - loss: 6.2794e-05
Epoch 13/50
32/32 - 0s - loss: 5.6360e-05
Epoch 14/50
32/32 - 0s - loss: 5.2891e-05
Epoch 15/50
32/32 - 0s - loss: 4.4200e-05
Epoch 16/50
32/32 - 0s - loss: 5.2062e-05
Epoch 17/50
32/32 - 0s - loss: 3.3509e-05
Epoch 18/50
32/32 - 0s - loss: 3.4012e-05
Epoch 19/50
32/32 - 0s - loss: 2.9005e-05
Epoch 20/50
32/32 - 0s - loss: 3.3826e-05
Epoch 21/50
32/32 - 0s - loss: 3.3222e-05
Epoch 22/50
32/32 - 0s - loss: 3.1733e-05
Epoch 23/50
32/32 - 0s - 

# Visualize training progress

In [11]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import *

RUN_NAME = "run 5 with 5 nodes"

training_data_df = pd.read_csv("sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

# Define the model
model = Sequential()
model.add(Dense(5, input_dim=9, activation='relu', name='layer_1'))
model.add(Dense(100, activation='relu', name='layer_2'))
model.add(Dense(50, activation='relu', name='layer_3'))
model.add(Dense(1, activation='linear', name='output_layer'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Create a TensorBoard logger
logger = keras.callbacks.TensorBoard(
    log_dir='logs/{}'.format(RUN_NAME),
    histogram_freq=5,
    write_graph=True
)

# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2,
    callbacks=[logger]
)

# Load the separate test data set
test_data_df = pd.read_csv("sales_data_test_scaled.csv")

X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

Epoch 1/50
32/32 - 1s - loss: 0.0409
Epoch 2/50
32/32 - 0s - loss: 0.0270
Epoch 3/50
32/32 - 0s - loss: 0.0236
Epoch 4/50
32/32 - 0s - loss: 0.0206
Epoch 5/50
32/32 - 0s - loss: 0.0176
Epoch 6/50
32/32 - 0s - loss: 0.0154
Epoch 7/50
32/32 - 0s - loss: 0.0136
Epoch 8/50
32/32 - 0s - loss: 0.0113
Epoch 9/50
32/32 - 0s - loss: 0.0098
Epoch 10/50
32/32 - 0s - loss: 0.0081
Epoch 11/50
32/32 - 0s - loss: 0.0066
Epoch 12/50
32/32 - 0s - loss: 0.0050
Epoch 13/50
32/32 - 0s - loss: 0.0039
Epoch 14/50
32/32 - 0s - loss: 0.0033
Epoch 15/50
32/32 - 0s - loss: 0.0023
Epoch 16/50
32/32 - 0s - loss: 0.0018
Epoch 17/50
32/32 - 0s - loss: 0.0014
Epoch 18/50
32/32 - 0s - loss: 0.0010
Epoch 19/50
32/32 - 0s - loss: 7.7143e-04
Epoch 20/50
32/32 - 0s - loss: 6.4499e-04
Epoch 21/50
32/32 - 0s - loss: 5.3022e-04
Epoch 22/50
32/32 - 0s - loss: 4.6682e-04
Epoch 23/50
32/32 - 0s - loss: 4.1241e-04
Epoch 24/50
32/32 - 0s - loss: 3.7466e-04
Epoch 25/50
32/32 - 0s - loss: 3.3186e-04
Epoch 26/50
32/32 - 0s - loss: 

# Exporting Google Cloud-compatible models

In [21]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import *
import tensorflow as tf

training_data_df = pd.read_csv("sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

# Define the model
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu', name='layer_1'))
model.add(Dense(100, activation='relu', name='layer_2'))
model.add(Dense(50, activation='relu', name='layer_3'))
model.add(Dense(1, activation='linear', name='output_layer'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Create a TensorBoard logger
logger = keras.callbacks.TensorBoard(
    log_dir='logs',
    histogram_freq=5,
    write_graph=True
)

# Train the model
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2,
    callbacks=[logger]
)

# Load the separate test data set
test_data_df = pd.read_csv("sales_data_test_scaled.csv")

X_test = test_data_df.drop('total_earnings', axis=1).values
Y_test = test_data_df[['total_earnings']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))

model_builder = tf.compat.v1.saved_model.Builder("exported_model")

if tf.executing_eagerly():
   tf.compat.v1.disable_eager_execution()

inputs = {
    'input': tf.compat.v1.saved_model.utils.build_tensor_info(model.input)
}
outputs = {
    'earnings': tf.compat.v1.saved_model.utils.build_tensor_info(model.output)
}

signature_def = tf.compat.v1.saved_model.signature_def_utils.build_signature_def(
    inputs=inputs,
    outputs=outputs,
    method_name=tf.compat.v1.saved_model.signature_constants.PREDICT_METHOD_NAME
)

model_builder.add_meta_graph_and_variables(
    tf.compat.v1.keras.backend.get_session(),
    tags=[tf.compat.v1.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.compat.v1.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
    }
)

model_builder.save()


Train on 1000 samples
Epoch 1/50
1000/1000 - 0s - loss: 0.0149
Epoch 2/50
1000/1000 - 0s - loss: 0.0015
Epoch 3/50
1000/1000 - 0s - loss: 6.8470e-04
Epoch 4/50
1000/1000 - 0s - loss: 3.8683e-04
Epoch 5/50
1000/1000 - 0s - loss: 2.1991e-04
Epoch 6/50
1000/1000 - 0s - loss: 1.5910e-04
Epoch 7/50
1000/1000 - 0s - loss: 1.1701e-04
Epoch 8/50
1000/1000 - 0s - loss: 9.8119e-05
Epoch 9/50
1000/1000 - 0s - loss: 1.0681e-04
Epoch 10/50
1000/1000 - 0s - loss: 7.8834e-05
Epoch 11/50
1000/1000 - 0s - loss: 5.8388e-05
Epoch 12/50
1000/1000 - 0s - loss: 5.4022e-05
Epoch 13/50
1000/1000 - 0s - loss: 4.6347e-05
Epoch 14/50
1000/1000 - 0s - loss: 4.7735e-05
Epoch 15/50
1000/1000 - 0s - loss: 4.4843e-05
Epoch 16/50
1000/1000 - 0s - loss: 5.0076e-05
Epoch 17/50
1000/1000 - 0s - loss: 4.6011e-05
Epoch 18/50
1000/1000 - 0s - loss: 4.0474e-05
Epoch 19/50
1000/1000 - 0s - loss: 3.5247e-05
Epoch 20/50
1000/1000 - 0s - loss: 2.9812e-05
Epoch 21/50
1000/1000 - 0s - loss: 3.2258e-05
Epoch 22/50
1000/1000 - 0s - 

AttributeError: module 'tensorflow' has no attribute 'Session'